In [22]:
import json
import pandas as pd
import os
import shutil
from glob import glob
import zipfile
import numpy as np
from tqdm import tqdm
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
QV_THRESHOLD = 20 

In [21]:
class_list={0:"Tumor epithelial",
1:"Basal/Myoepithelial/Smooth muscle",
2:"Fibroblast",
3:"Endothelial",
4:"T cell",
5:"B cell",
6:"Plasma cell",
7:"Myeloid",
8:"Adipocyte",
9:"Mast cell",
10:"Other/Unknown"}
marker_genes = {
    'Tumor epithelial': [b'EPCAM', b'KRT14', b'KRT5', b'KRT23', b'ERBB2', b'MKI67', b'GATA3', b'KRT18', b'KRT19'],
    'Basal/Myoepithelial/Smooth muscle': [b'ACTA2', b'MYH11', b'TAGLN', b'KRT14', b'KRT5', b'MYL9'],
    'Fibroblast': [b'PDGFRA', b'PDGFRB', b'DPT', b'LUM', b'SFRP1', b'FBLN1', b'COL1A1', b'DCN', b'SFRP4'],
    'Endothelial': [b'PECAM1', b'KDR', b'CD93', b'EGFL7', b'VWF', b'CD34', b'CLEC14A'],
    'T cell': [b'CD3D', b'CD3E', b'CD3G', b'NKG7', b'GZMA', b'CCL5', b'TRAC', b'TCF7', b'LTB', b'IL2RG'],
    'B cell': [b'CD79A', b'CD79B', b'MS4A1', b'MZB1', b'CD19', b'PAX5'],
    'Plasma cell': [b'MZB1', b'SDC1', b'JCHAIN', b'IGLC2', b'IGHG1'],
    'Myeloid': [b'CD68', b'CD14', b'CD163', b'MRC1', b'C1QA', b'AIF1', b'S100A8', b'CD86', b'ITGAX'],
    'Adipocyte': [b'ADIPOQ', b'LEP', b'PLIN1', b'FABP4', b'LPL', b'PPARG'],
    'Mast cell': [b'TPSAB1', b'TPSB2', b'CPA3', b'KIT', b'MS4A2'],
    'Other/Unknown': [] 
}

In [4]:
xenium_annotation_list=glob('../../data/Brest_spatialTranscriptome/xenium_seg/*_xenium_nucleus_seg.parquet')
xenium_wsi_list = [f.replace("xenium_seg/", "wsis/") for f in xenium_annotation_list]
xenium_wsi_list = [f.replace("_xenium_nucleus_seg.parquet", ".tif") for f in xenium_wsi_list]
xenium_metadata_list = [f.replace("wsis/", "metadata/") for f in xenium_wsi_list]
xenium_metadata_list = [f.replace(".tif", ".json") for f in xenium_metadata_list]
save_path='../../data/Brest_spatialTranscriptome/preprocessed_xenium/'
for i in tqdm(range(len(xenium_annotation_list))):
    zip_path = xenium_annotation_list[i]
    wsi_path = xenium_wsi_list[i]
    create_dir(save_path+'wsis/')
    create_dir(save_path+'metadata/')
    shutil.copy(wsi_path, f'{save_path}wsis/{os.path.basename(wsi_path)}')
    shutil.copy(xenium_metadata_list[i], f'{save_path}metadata/{os.path.basename(xenium_metadata_list[i])}')


100%|██████████| 9/9 [02:44<00:00, 18.27s/it]


In [30]:
import pandas as pd

xenium_transcripts_list=[f.replace("xenium_seg/", "transcripts/") for f in xenium_annotation_list]
xenium_transcripts_list=[f.replace("_xenium_nucleus_seg.parquet", "_transcripts.parquet") for f in xenium_transcripts_list]
i=0
xenium_transcript_path = xenium_transcripts_list[i]
df_transcript = pd.read_parquet(xenium_transcript_path)
df_filtered = df_transcript[df_transcript['qv'] > QV_THRESHOLD].copy()

df_filtered = df_filtered[~df_filtered['feature_name'].str.decode('utf-8').str.contains('BLANK|NegControl|antisense', case=False, na=False)]
xenium_annotation_path=xenium_annotation_list[i]
df_seg = pd.read_parquet(xenium_annotation_path)
df=pd.DataFrame(columns=['x1','y1','x2','y2','class_name'])
j=0


In [27]:
df_filtered

,transcript_id,cell_id,overlaps_nucleus,feature_name,x_location,y_location,z_location,qv,fov_name,nucleus_distance,geometry,he_x,he_y
0,281474976710656,b'aagigajm-1',0,b'RUNX1',15.260350,486.198669,8.657274,37.950714,b'A1',0.000000,b'\x01\x01\x00\x00\x00\xc3H\x84\x81\xb7\x0f\xd...,18494.867280,4866.680232
2,281474976710658,b'aahpibdb-1',0,b'TCIM',18.547781,813.857788,13.415476,40.000000,b'A1',2.795314,b'\x01\x01\x00\x00\x00xP\xa6\xfb\xbd\xe4\xd0@\...,17298.968484,4879.829254
3,281474976710659,b'aagnjbkb-1',0,b'LUM',17.891180,425.405334,8.532911,40.000000,b'A1',1.741180,b'\x01\x01\x00\x00\x00\x9cc\xf0\x974G\xd2@\xdf...,18716.821774,4876.253289
4,281474976710660,b'aahfmlaj-1',0,b'LUM',20.031204,299.748718,9.812729,40.000000,b'A1',5.207764,b'\x01\x01\x00\x00\x00\xbe\xeec\x07\xbe\xb9\xd...,19174.969201,4883.839602
6,281474976710662,b'aajeflbb-1',1,b'RUNX1',20.910904,174.818115,10.379794,38.377777,b'A1',0.000000,b'\x01\x01\x00\x00\x00\xa5\xa7L\xd3\xb9+\xd3@\...,19630.903522,4886.556190
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15643046,281779919692264,b'odpcchhg-1',1,b'GZMA',5439.657715,4906.468750,63.081654,40.000000,b'F9',0.000000,b'\x01\x01\x00\x00\x00\x90C\x1f+\xe0y\xa2@\x94...,2364.937829,24675.318501
15643047,281779919692265,b'oedgkecm-1',0,b'DST',5438.693848,4514.345703,62.915199,40.000000,b'F9',11.562809,b'\x01\x01\x00\x00\x00%\xb2\x17\xac\xb7\xa7\xa...,3795.858735,24671.180282
15643048,281779919692266,b'oelnomma-1',1,b'USP53',5439.778320,4860.086914,65.668259,40.000000,b'F9',0.000000,b'\x01\x01\x00\x00\x00\x08\x9a\x15\x84\x81\xcc...,2534.252961,24675.978177
15643049,281779919692274,b'odikggam-1',1,b'APOBEC3B',5439.949219,4692.814453,65.748428,31.568277,b'F9',0.000000,b'\x01\x01\x00\x00\x00\xdd\x9c\xb3`*\x92\xa8@]...,3145.082769,24676.026767


In [29]:
len(df_seg)

142272